In [84]:
import argparse

parser = argparse.ArgumentParser(description='Simple Network.')
args, unknown_args = parser.parse_known_args()

In [85]:
import os, random, sys
import numpy as np

import tensorflow as tf
from tensorflow.python.ops import rnn, rnn_cell
slim = tf.contrib.slim

In [86]:
import sklearn.metrics as metrics
import Shared

In [87]:
y = np.load('cache/y.original.npy')
yt = np.load('cache/yt.original.npy')

In [188]:
models = [
    'alexnet.distracted_driver.original.1',
    'alexnet.distracted_driver.segmented.1',
#     'alexnet.distracted_driver.face.1',
#     'alexnet.distracted_driver.hands.1',
#     'alexnet.distracted_driver.hands_and_face.1',
#     'inceptionV3.distracted_driver.original.1',
#     'inceptionV3.distracted_driver.segmented.1',
#     'inceptionV3.distracted_driver.face.1',
#     'inceptionV3.distracted_driver.hands.1',
#     'inceptionV3.distracted_driver.hands_and_face.1',
]

In [189]:
X = [ np.load("Prediction.X.{}.npy".format(model)) for model in models ]
Xt = [ np.load("Prediction.Xt.{}.npy".format(model)) for model in models ]

In [190]:
for model in Xt:
    print("{:.4f}\t{:.2f}".format(metrics.log_loss(yt, model), 100 * metrics.accuracy_score(yt.argmax(axis=1), model.argmax(axis=1))))

0.3909	93.65
0.3446	93.60


In [191]:
ens = lambda x: reduce(lambda prev, curr: prev+curr, x) / len(x)

In [192]:
print(metrics.log_loss(y, ens(X)))
print(metrics.log_loss(yt, ens(Xt)), metrics.accuracy_score(yt.argmax(axis=1), ens(Xt).argmax(axis=1)))

0.00246645230749
(0.27270362937570819, 0.94296929115677675)


In [178]:
from random import randint, random, sample
from operator import add

def individual(length):
    'Create a member of the population.'
    return [ random() for x in xrange(length) ]

def population(count, length):
    """
    Create a number of individuals (i.e. a population).

    count: the number of individuals in the population
    length: the number of values per individual

    """
    return [ individual(length) for x in xrange(count) ]

def fitness(individual, portion=0.5):
    """
    Determine the fitness of an individual. Higher is better.

    individual: the individual to evaluate
    target: the target number individuals are aiming for
    """
    size = X[0].shape[0]
    idx = sample(range(0, size), int(portion*size))
    total = reduce(add, individual, 0)
    ens = reduce(lambda prev, curr: prev + curr[0] * curr[1][idx], zip(individual, X), np.zeros(X[0][idx].shape)) / total
    return metrics.log_loss(y[idx], ens)

def grade(pop):
    'Find average fitness for a population.'
    summed = reduce(add, (fitness(x) for x in pop))
    return summed / (len(pop) * 1.0)

def evolve(pop, retain=0.2, random_select=0.1, mutate=0.05):
    graded = [ (fitness(x), x) for x in pop]
    graded = [ x[1] for x in sorted(graded)]
    retain_length = int(len(graded)*retain)
    parents = graded[:retain_length]
    # randomly add other individuals to
    # promote genetic diversity
    for individual in graded[retain_length:]:
        if random_select > random():
            parents.append(individual)
    # mutate some individuals
    for individual in parents:
        if mutate > random():
            pos_to_mutate = randint(0, len(individual)-1)
            # this mutation is not ideal, because it
            # restricts the range of possible values,
            # but the function is unaware of the min/max
            # values used to create the individuals,
            individual[pos_to_mutate] = random()
    # crossover parents to create children
    parents_length = len(parents)
    desired_length = len(pop) - parents_length
    children = []
    while len(children) < desired_length:
        male = randint(0, parents_length-1)
        female = randint(0, parents_length-1)
        if male != female:
            male = parents[male]
            female = parents[female]
            half = len(male) / 2
            child = male[:half] + female[half:]
            children.append(child)
    parents.extend(children)
    return parents

In [182]:
# Example usage
p_count = 50
p = population(p_count, len(X))
fitness_history = [grade(p),]
for i in xrange(10):
    p = evolve(p)
    fitness_history.append(grade(p))
    Shared.update_screen("\rStep {}: Min Loss={}".format(i, np.min(fitness_history)))

Step 9: Min Loss=0.000259512942775

In [183]:
graded = [ (fitness(x, portion=1.0), x) for x in p]
graded = [ x for x in sorted(graded)]

In [184]:
graded[-1]

(0.00069354553659086793,
 [0.5055497129632993,
  0.03864812937582929,
  0.9661346080012782,
  0.9688160444116005])

In [185]:
weights = graded[-1][1]
total = reduce(add, weights, 0)
probs_ens = reduce(lambda prev, curr: prev + curr[0] * curr[1], zip(weights, Xt), np.zeros(Xt[0].shape)) / total

In [186]:
print(metrics.log_loss(yt, probs_ens))

0.203550661114


In [187]:
print(metrics.accuracy_score(yt.argmax(axis=1), probs_ens.argmax(axis=1)))

0.954975756176


In [119]:
graded[-1]

(0.0012756510877630141,
 [0.18309921362793558,
  0.4326389772065735,
  0.10991134113843704,
  0.01194613485192364,
  0.00971146230693376,
  0.9320650338477107,
  0.8376795455837012,
  0.08035432265243636,
  0.39423251988088026,
  0.7441139851401929])

In [158]:
weights = graded[-1][1]
weights = [0.18309921362793558,
  0.4326389772065735,
  0.10991134113843704,
  0.01194613485192364,
  0.00971146230693376,
  0.9320650338477107,
  0.8376795455837012,
  0.08035432265243636,
  0.39423251988088026,
  0.7441139851401929]
total = reduce(add, weights, 0)
probs_ens = reduce(lambda prev, curr: prev + curr[0] * curr[1], zip(weights, Xt), np.zeros(Xt[0].shape)) / total

In [159]:
print(metrics.log_loss(yt, probs_ens))

0.157597373037


In [160]:
print(metrics.accuracy_score(yt.argmax(axis=1), probs_ens.argmax(axis=1)))

0.959824520896


#### Make Submission

In [ ]:
import pandas as pd

In [ ]:
weights = graded[-1][1]
total = reduce(add, weights, 0)
probs_ens = reduce(lambda prev, curr: prev + curr[0] * curr[1], zip(weights, Xt), np.zeros(Xt[0].shape)) / total

In [38]:
submission = np.hstack((np.arange(1, probs_ens.shape[0]+1, 1).reshape(-1, 1), probs_ens[:, 0].reshape(-1,1)))
df_submission = pd.DataFrame(submission, columns=["id", "label"])
df_submission["id"] = df_submission["id"].astype(np.int)
df_submission.to_csv("submission_{}.csv".format("deep_ensemble_ga_weights_least_fit"), index=False)

### Statistics

In [149]:
import numpy as np
import matplotlib.pyplot as plt

plt.rc('text', usetex=False)
plt.rc('font', family='serif')

In [150]:
import seaborn as sn

In [151]:
import itertools

In [162]:
metrics.confusion_matrix(y_pred=Xt[0].argmax(axis=1), y_true=yt.argmax(axis=1))

array([[857,   2,   1,   4,   2,  10,   8,  12,   0,  26],
       [  0, 307,  10,   0,   7,   0,   2,   0,   0,   0],
       [  4,  13, 322,   2,   0,   0,   0,   0,   0,   0],
       [ 12,   7,   0, 466,   4,   1,   1,   1,   1,   1],
       [  1,   2,   0,  16, 283,   0,   4,   0,   0,   0],
       [ 17,   0,   0,   1,   0, 281,   3,   1,   0,   2],
       [ 10,   0,   1,   2,   0,   1, 385,   4,   0,   0],
       [ 14,   1,   0,   0,   1,   0,   3, 276,   1,   5],
       [ 14,   0,   0,   1,   3,   0,   3,   2, 263,   4],
       [ 20,   0,   0,   0,   0,   0,   4,   1,   2, 616]])

In [163]:
metrics.confusion_matrix(y_pred=Xt[1].argmax(axis=1), y_true=yt.argmax(axis=1))

array([[867,   1,   2,   6,   2,   6,   6,   6,   3,  23],
       [  4, 301,   8,   2,   5,   1,   5,   0,   0,   0],
       [  2,  10, 326,   0,   0,   3,   0,   0,   0,   0],
       [ 11,   4,   1, 466,   6,   2,   1,   0,   1,   2],
       [  1,   3,   0,  12, 287,   0,   2,   0,   1,   0],
       [ 10,   0,   1,   0,   0, 288,   3,   0,   2,   1],
       [ 12,   1,   0,   1,   0,   4, 378,   2,   2,   3],
       [ 16,   0,   0,   0,   1,   0,   2, 275,   0,   7],
       [ 12,   0,   0,   2,   1,   0,   5,   5, 258,   7],
       [ 16,   0,   1,   2,   1,   1,   8,   3,   3, 608]])

In [165]:
metrics.confusion_matrix(y_pred=Xt[2].argmax(axis=1), y_true=yt.argmax(axis=1))

array([[799,   6,   4,  19,   0,  13,  20,  14,   9,  38],
       [  9, 284,  12,   4,   5,   0,   5,   3,   2,   2],
       [  5,  17, 303,   1,   1,   3,   7,   1,   1,   2],
       [ 23,   9,   3, 426,  10,   6,   8,   1,   2,   6],
       [  4,   5,   2,  17, 267,   0,   4,   4,   1,   2],
       [ 16,   3,   0,   3,   0, 259,  10,   4,   6,   4],
       [ 31,   2,   4,   7,   0,   7, 316,   7,  10,  19],
       [ 27,   1,   2,   4,   6,   2,  14, 212,  22,  11],
       [ 20,   2,   3,   3,   1,   4,   9,  22, 210,  16],
       [ 24,   1,   1,   2,   1,   3,   8,  15,  14, 574]])

In [166]:
metrics.confusion_matrix(y_pred=Xt[3].argmax(axis=1), y_true=yt.argmax(axis=1))

array([[836,   4,  10,  10,   1,   3,  17,   2,   1,  38],
       [  6, 286,  17,   0,   4,   0,   5,   2,   0,   6],
       [ 15,  18, 302,   1,   0,   1,   0,   1,   1,   2],
       [ 13,   3,   0, 460,   4,   1,   6,   3,   0,   4],
       [  0,   5,   2,  14, 271,   2,  11,   0,   1,   0],
       [ 17,   0,   2,   0,   2, 272,   5,   2,   3,   2],
       [ 14,   3,   0,   2,   3,   0, 366,   4,   1,  10],
       [ 13,   2,   0,   4,   2,   0,   6, 268,   1,   5],
       [ 13,   2,   0,   4,   3,   0,  10,   2, 243,  13],
       [ 40,   2,   1,   0,   2,   1,  16,   2,   6, 573]])

In [167]:
metrics.confusion_matrix(y_pred=Xt[4].argmax(axis=1), y_true=yt.argmax(axis=1))

array([[827,   6,   5,   9,   4,   6,  25,  10,   7,  23],
       [ 10, 284,  14,   4,   4,   0,   3,   2,   3,   2],
       [  9,  12, 309,   2,   2,   1,   5,   0,   0,   1],
       [ 21,   2,   1, 442,   7,   2,   7,   5,   2,   5],
       [  6,   7,   2,   9, 270,   1,   4,   2,   2,   3],
       [ 20,   2,   1,   7,   1, 257,   3,   4,   4,   6],
       [ 27,   3,   4,   3,   9,   4, 316,   9,  13,  15],
       [ 15,   1,   4,   4,   3,   1,   8, 249,   8,   8],
       [ 17,   1,   2,   4,   4,   0,  11,  13, 219,  19],
       [ 31,   1,   2,   2,   2,   3,   7,   6,   8, 581]])

In [161]:
metrics.confusion_matrix(y_pred=probs_ens.argmax(axis=1), y_true=yt.argmax(axis=1))

array([[879,   0,   3,   6,   1,   4,   4,   8,   1,  16],
       [  1, 315,   4,   1,   3,   0,   1,   0,   1,   0],
       [  1,  11, 329,   0,   0,   0,   0,   0,   0,   0],
       [ 10,   3,   0, 475,   4,   0,   1,   0,   0,   1],
       [  0,   1,   0,  15, 290,   0,   0,   0,   0,   0],
       [ 13,   0,   0,   1,   0, 290,   0,   0,   0,   1],
       [  3,   0,   0,   1,   0,   3, 395,   1,   0,   0],
       [ 11,   0,   0,   0,   0,   0,   0, 287,   0,   3],
       [ 11,   0,   0,   0,   0,   0,   4,   1, 269,   5],
       [  9,   0,   0,   0,   0,   0,   3,   2,   1, 628]])

In [154]:
CLASSES = [
    "Drive Safe",
    "Text Left",
    "Talk Left",
    "Text Right",
    "Talk Right",
    "Adjust Radio",
    "Drink",
    "Hair & Makeup",
    "Reach Behind",
    "Talk Passenger"
]

In [155]:
plot_confusion_matrix(cm, CLASSES)

Confusion matrix, without normalization
[[857   2   1   4   2  10   8  12   0  26]
 [  0 307  10   0   7   0   2   0   0   0]
 [  4  13 322   2   0   0   0   0   0   0]
 [ 12   7   0 466   4   1   1   1   1   1]
 [  1   2   0  16 283   0   4   0   0   0]
 [ 17   0   0   1   0 281   3   1   0   2]
 [ 10   0   1   2   0   1 385   4   0   0]
 [ 14   1   0   0   1   0   3 276   1   5]
 [ 14   0   0   1   3   0   3   2 263   4]
 [ 20   0   0   0   0   0   4   1   2 616]]
